In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from scipy.stats import randint, uniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [12]:
df = pd.read_csv('Данные для построения модели, задача - Классификация - превышает ли значение SI значение 8.csv')
df

,HallKierAlpha,Chi1,SlogP_VSA6,Chi0,BCUT2D_CHGHI,MinPartialCharge,EState_VSA10,SMR_VSA7,SMR_VSA4,fr_allylic_oxid,...,SMR_VSA1,BCUT2D_MRLOW,BCUT2D_MWLOW,VSA_EState5,EState_VSA3,PEOE_VSA6,MinAbsPartialCharge,SMR_VSA5,MaxAbsEStateIndex,Target Feature
0,-0.66,13.127794,9.984809,20.208896,2.600532,-0.293526,0.000000,0.000000,43.480583,0,...,0.000000,0.193481,9.700470,1.764908,21.659962,54.384066,0.038844,105.750639,5.094096,1
1,-0.08,13.127794,0.000000,20.208896,2.614066,-0.313407,0.000000,0.000000,33.495774,0,...,0.000000,0.120322,9.689226,1.914377,21.659962,54.384066,0.012887,117.834321,3.961417,0
2,-0.08,14.595754,0.000000,23.794682,2.665274,-0.325573,0.000000,0.000000,33.495774,0,...,8.966062,-0.922902,9.681293,1.930720,21.659962,41.542423,0.094802,117.834321,2.627117,0
3,-0.66,13.627794,9.984809,20.916003,2.600529,-0.293526,0.000000,0.000000,43.480583,0,...,0.000000,0.193510,9.700497,1.769975,21.659962,60.804888,0.038844,112.171461,5.097360,1
4,-2.22,16.526773,58.515746,24.639617,2.602486,-0.257239,0.000000,59.657840,43.480583,0,...,0.000000,0.193687,9.700386,1.605178,10.829981,65.807891,0.062897,86.488175,5.150510,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,-1.65,14.097861,11.649125,21.637464,2.726237,-0.468587,14.383612,11.649125,51.752408,2,...,23.857337,-0.196385,9.344314,-0.405436,11.835812,38.841158,0.317890,66.219879,12.934891,0
997,-2.18,16.402391,11.649125,25.499271,2.725543,-0.467493,19.178149,11.649125,51.752408,2,...,28.651875,-0.187625,9.343622,-1.300354,5.917906,45.764895,0.327562,79.185457,13.635345,0
998,-1.83,17.940396,11.649125,27.620591,2.725818,-0.467485,19.178149,11.649125,51.752408,2,...,28.651875,-0.187687,9.343613,-0.487671,12.338728,45.764895,0.327887,78.682541,13.991690,0
999,-2.45,17.309003,0.000000,27.921921,2.730109,-0.468755,23.972686,0.000000,51.752408,0,...,42.920138,-0.225309,9.364015,-6.848660,19.262465,27.192033,0.312509,66.219879,13.830180,0


In [13]:
X, y = df.drop(['Target Feature'], axis=1), df['Target Feature']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
y.value_counts()

Target Feature
0    644
1    357
Name: count, dtype: int64

Классы не сбалансированны, произведем процесс балансировки классов

In [16]:
smote = SMOTE(sampling_strategy='auto', k_neighbors=10, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [17]:
X, y = X_resampled, y_resampled
y.value_counts()

Target Feature
1    644
0    644
Name: count, dtype: int64

Классы сбалансированны, можно приступать обучения моделей

In [18]:
class BestClassifierModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели классификации
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'F1-score',
            'value': 0.7
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LogisticRegression(max_iter=1000, random_state=42),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': [0.01, 0.1, 1, 10],
                    'penalty': ['l1', 'l2'],
                    'solver': ['saga']
                },
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': KNeighborsClassifier(),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'n_neighbors': range(3, 15),
                    'weights': ['uniform', 'distance'],
                    'p': [1, 2]  # 1 = манхэттенское расстояние, 2 = евклидово расстояние
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestClassifier(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': ''
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVC(random_state=42),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            
            self.models[model]['metrics']['Accurac'] = accuracy_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Precision'] = precision_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Recall'] = recall_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['F1-score'] = f1_score(y, self.models[model]['data']['y_pred'])
            
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"Accuracy: {self.models[model]['metrics']['Accurac']:.4f}")
            print(f"Precision: {self.models[model]['metrics']['Precision']:.4f}")
            print(f"Recall: {self.models[model]['metrics']['Recall']:.4f}")
            print(f"F1-score: {self.models[model]['metrics']['F1-score']:.4f}")
            print()

            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return result

In [19]:
best_regression_model = BestClassifierModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LogisticRegression(max_iter=1000, random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...


g:\PythonProject\Educational_Folder\gihub_ml\Coursework_Classik_ML\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\gihub_ml\Coursework_Classik_ML\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\gihub_ml\Coursework_Classik_ML\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\gihub_ml\Coursework_Classik_ML\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\gihub_ml\Coursework_Classik_ML\venv\Lib\site-packages\sklearn\li

Лучшие параметры для модели LogisticRegression(max_iter=1000, random_state=42): {'C': 1, 'penalty': 'l1', 'solver': 'saga'}

Подбор гипперпараметров для модели KNeighborsClassifier() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели KNeighborsClassifier(): {'n_neighbors': 4, 'p': 1, 'weights': 'distance'}

Подбор гипперпараметров для модели RandomForestClassifier(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestClassifier(random_state=42): {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 67}

Подбор гипперпараметров для модели SVC(random_state=42) методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели SVC(ran

In [20]:
best_regression_model.get_best_model()

Обучаем модель LogisticRegression(max_iter=1000, random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LogisticRegression(max_iter=1000, random_state=42):
Accuracy: 0.6357
Precision: 0.6117
Recall: 0.6738
F1-score: 0.6412

Обучаем модель KNeighborsClassifier() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели KNeighborsClassifier():
Accuracy: 0.7597
Precision: 0.7304
Recall: 0.7968
F1-score: 0.7621

Обучаем модель RandomForestClassifier(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestClassifier(random_state=42):
Accuracy: 0.7700
Precisi

In [21]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели KNeighborsClassifier(n_neighbors=4, p=1, weights='distance')
Перед формированияем прогноза, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаю формированию прогноза...
Предсказание сформировано


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]